In [1]:
import warnings

warnings.filterwarnings("ignore")
import tensorflow as tf

tf.get_logger().setLevel("ERROR")

In [2]:
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from keras.models import load_model

Using TensorFlow backend.


In [3]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc

rc("animation", html="html5")

In [4]:
from dataset import suny_international
from animate import plot_daily
from utility import split_sequence

In [6]:
df = suny_international.load_data()
df_train, df_test = suny_international.train_test_split(df)
plot_daily(df['GHI'],save='media/GHI.mp4', index=df.index)


In [11]:
from IPython.display import HTML
HTML("""
<div align="middle">
<video width="80%"  src="media/plot_simple.mp4" type="video/mp4" controls>
</video></div>""")

In [ ]:
X = np.array(df_test.index.minute + df_test.index.hour * 60).reshape(-1, 1)
y = np.array(df_test["GHI"])
model = load_model("models/simple_dnn_daily.h5")
y_pred = model.predict(X)
plot_daily(y, y_pred.ravel(), index=df_test.index)

In [ ]:
X = np.array([df_test.index.minute + df_test.index.hour * 60, df_test.index.month]).T
y = np.array(df_test["GHI"])
model = load_model("models/simple_dnn_monthly.h5")
y_pred = model.predict(X)
plot_daily(y, y_pred.ravel(), index=df_test.index)

In [ ]:
X = np.array(
    [
        df_test.index.minute + df_test.index.hour * 60,
        df_test.index.month,
        *(df_test[factor] for factor in suny_international.atmospheric_factors),
    ]
).T
y = np.array(df_test["GHI"])
model = load_model("models/simple_dnn_atmospheric.h5")
y_pred = model.predict(X)
plot_daily(y, y_pred.ravel(), index=df_test.index)

In [ ]:
n_features = 1
n_steps = 20
X, y = split_sequence(df_test["GHI"], n_steps)
X = X.reshape((X.shape[0], X.shape[1], n_features))
model = load_model("models/simple_rnn.h5")
y_pred = model.predict(X)
plot_daily(y, y_pred.ravel(), index=df_test.index[n_steps:])